In [7]:
import sympy as sp
from sympy.physics.quantum import TensorProduct
import numpy as np
from qutip import qsave,qload
digits=400
def norm(vec):
    i=0
    for ele in vec:
        i = abs(ele)**2 + i
    return i
def annihilation(n):
    def anni_(i, j):
        if j-i == 1:
            return sp.sqrt(j)
        else:
            return 0
    return sp.Matrix(n, n, anni_)
def creation(n):
    def crea_(i, j):
        if i-j == 1:
            return sp.sqrt(i)
        else:
            return 0
    return sp.Matrix(n, n, crea_)
def get_H():
    dim = 20
    Q_dim=6
    g=0.1*2*sp.pi
    anharmonicity =-0.225
    a_dag = creation(dim)
    a = annihilation(dim)
    b_dag=creation(Q_dim)
    b=annihilation(Q_dim)
    A=TensorProduct(a,sp.eye(Q_dim))
    A_dag=TensorProduct(a_dag,sp.eye(Q_dim))
    B=TensorProduct(sp.eye(dim),b)
    B_dag=TensorProduct(sp.eye(dim),b_dag)
    H_trans = 1 / 2 * anharmonicity * ((b_dag*b)*(b_dag*b-sp.eye(Q_dim)))
    H0=g*(TensorProduct(a_dag,b)+TensorProduct(a,b_dag))+TensorProduct(sp.eye(dim),H_trans)
    H=-1j*(H0+0.5*2*sp.pi*(B_dag*B+B+B_dag+1j*(B-B_dag)))
    vec=1/sp.sqrt(dim*Q_dim)*sp.ones(dim*Q_dim,1)
    vec=vec.evalf(digits)
    return H.evalf(digits),vec
def get_auxiliary(t):
    dim = 20
    Q_dim=6
    g=0.1*2*sp.pi
    anharmonicity =-0.225
    a_dag = creation(dim)
    a = annihilation(dim)
    b_dag=creation(Q_dim)
    b=annihilation(Q_dim)
    A=TensorProduct(a,sp.eye(Q_dim))
    A_dag=TensorProduct(a_dag,sp.eye(Q_dim))
    B=TensorProduct(sp.eye(dim),b)
    B_dag=TensorProduct(sp.eye(dim),b_dag)
    H_trans = 1 / 2 * anharmonicity * ((b_dag*b)*(b_dag*b-sp.eye(Q_dim)))
    H0=g*(TensorProduct(a_dag,b)+TensorProduct(a,b_dag))+TensorProduct(sp.eye(dim),H_trans)
    H=-1j*(H0+0.5*2*sp.pi*(B_dag*B+B+B_dag+1j*(B-B_dag)))
    A=sp.Matrix([[t*H,-1j*(B+B_dag)],[0*H,t*H]])
    vec=1/sp.sqrt(dim*Q_dim)*sp.ones(dim*Q_dim,1)
    vec=sp.Matrix([0*vec,vec])
    return A.evalf(digits),vec.evalf(digits)
def expm(H,vec):
    b=vec
    c=vec
    j=1
    while(1):
        b=(H*b/j).evalf(digits)
        c=(c+b).evalf(digits)
        if max(abs(np.array(b,dtype=np.complex128))) <= 10**-100:
            break
        j=j+1
    return c

In [2]:
ts = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
ref = {"state_propagator":[],"state_propagator_der":[],"gate_propagator":[],"gate_propagator_der":[]}

In [3]:
for t in ts:
    H,vec = get_H()
    ref["state_propagator_der"].append(expm(t*H,vec))

KeyboardInterrupt: 

In [ ]:
for t in ts:
    H,vec = get_auxiliary(1)
    ref["state_propagator_der"].append(expm(t*H,vec))

In [4]:
digits=300
def get_H(N):
    control=get_control(N)
    control_pra=0.5*2*sp.pi*sp.ones(len(control),1)
    H_control=control_H(control_pra,control)
    H=-1j*(H_control+get_int(N)*0.1*2*sp.pi)
    vec=1/sp.sqrt(2**N)*sp.ones(2**N,1)
    return H.evalf(digits),vec.evalf(digits)
def get_control(N,):
    sigmap=creation(2,)
    sigmam =annihilation(2,)
    sigmap=sigmap
    sigmam=sigmam
    sigmax=sigmap+sigmam
    sigmay=-1j*sigmap+1j*sigmam
    control=[]
    if N==1:
        control.append(TensorProduct(sigmax, sp.eye(2 ** (N - 1))))
        control.append(TensorProduct(sigmay, sp.eye(2 ** (N - 1))))
        return control
    else:
        a=sp.eye(2**(N-1))
        control.append(TensorProduct(sigmax,a))
        control.append(TensorProduct(sigmay, sp.eye(2 ** (N - 1))))
        for i in range(1,N-1):
            control.append(TensorProduct(TensorProduct(sp.eye(2**i),sigmax), sp.eye(2 ** (N - 1-i))))
            control.append(TensorProduct(TensorProduct(sp.eye(2 ** i), sigmay), sp.eye(2 ** (N - 1 - i))))
        control.append(TensorProduct(sp.eye(2**(N-1)),sigmax))
        control.append(TensorProduct(sp.eye(2**(N-1)),sigmay))
    return control
def control_H(control,H_control):
    H=0*H_control[0]
    for i in range(len(control)):
        H=H+control[i]*H_control[i]
    return H
def get_int(N):
    sigmap=creation(2)
    sigmam = annihilation(2)
    sigmaz=sigmap*(sigmam)
    H0=sp.zeros(2**N)
    SIGMAZ=TensorProduct(sigmaz,sigmaz)
    H0=H0+TensorProduct(SIGMAZ,sp.eye(2**(N-2)))+TensorProduct(sp.eye(2**(N-2)),SIGMAZ)
    for i in range(1,N-2):
        H0=H0+TensorProduct(TensorProduct(sp.eye(2**i),SIGMAZ),sp.eye(2 ** (N - 2 - i)))
    return H0
def get_auxiliary(N,t):
    H,vec = get_H(N)
    control=get_control(N)
    vec = sp.Matrix([0*vec,vec])
    A = sp.Matrix([[t*H,-1j*control[3]],[0*H,t*H]])
    return A.evalf(digits), vec.evalf(digits)

In [260]:
N=7
for t in ts:
    H,vec = get_H(N)
    ref["gate_propagator"].append(expm(t*H,vec))


In [5]:
qsave(ref,'reference')

[41.91319078]
[8790.62303193]
[1169497.22362053]
[1.22428819e+08]
[1.05985136e+10]
[7.69075743e+11]
[4.88056681e+13]
[2.71424181e+15]
[1.35215489e+17]
[6.06539255e+18]
[2.47854254e+20]
[9.28770108e+21]
[3.21267368e+23]
[1.03243848e+25]
[3.09478055e+26]
[8.70240674e+27]
[2.30137735e+29]
[5.75177563e+30]
[1.36084744e+32]
[3.06071945e+33]
[6.55163988e+34]
[1.33948494e+36]
[2.61790226e+37]
[4.90599644e+38]
[8.82135117e+39]
[1.52590409e+41]
[2.54049851e+42]
[4.08047649e+43]
[6.32524022e+44]
[9.48184952e+45]
[1.37500484e+47]
[1.93232826e+48]
[2.6323771e+49]
[3.48166319e+50]
[4.47205847e+51]
[5.58617673e+52]
[6.78748081e+53]
[8.03207671e+54]
[9.25902177e+55]
[1.04088299e+57]
[1.14136737e+58]
[1.22198999e+59]
[1.27764886e+60]
[1.3057067e+61]
[1.3045172e+62]
[1.27518922e+63]
[1.21982447e+64]
[1.16166647e+65]
[1.08881146e+66]
[9.99823287e+66]
[8.9959812e+67]
[7.93635503e+68]
[6.86590115e+69]
[5.8283147e+70]
[4.85528804e+71]
[3.97149853e+72]
[3.1902108e+73]
[2.51784686e+74]
[1.95273215e+75]
[1.48

[1.81892656e+193]
[1.76259068e+193]
[1.70432645e+193]
[1.6444517e+193]
[1.58328275e+193]
[1.52113182e+193]
[1.45830451e+193]
[1.39509748e+193]
[1.33179637e+193]
[1.26867386e+193]
[1.20598804e+193]
[1.14398095e+193]
[1.08287743e+193]
[1.02288417e+193]
[9.64188991e+192]
[9.06960449e+192]
[8.51347565e+192]
[7.97479834e+192]
[7.45467421e+192]
[6.95401544e+192]
[6.4735503e+192]
[6.01383016e+192]
[5.57523785e+192]
[5.15799714e+192]
[4.76218308e+192]
[4.38773313e+192]
[4.03445868e+192]
[3.70205703e+192]
[3.39012344e+192]
[3.09816323e+192]
[2.82560369e+192]
[2.57180574e+192]
[2.33607515e+192]
[2.11767333e+192]
[1.91582749e+192]
[1.72974016e+192]
[1.55859802e+192]
[1.40158003e+192]
[1.25786475e+192]
[1.12663694e+192]
[1.0070934e+192]
[8.98448007e+191]
[7.99936097e+191]
[7.10818118e+191]
[6.30382617e+191]
[5.57948615e+191]
[4.92867404e+191]
[4.34523816e+191]
[3.82337007e+191]
[3.35760819e+191]
[2.94283749e+191]
[2.5742859e+191]
[2.24751788e+191]
[1.95842547e+191]
[1.70321741e+191]
[1.47840663e+1

[1.27264687e+111]
[6.2062401e+110]
[3.02328055e+110]
[1.47115373e+110]
[7.15101846e+109]
[3.47223e+109]
[1.68414827e+109]
[8.1598827e+108]
[3.94929676e+108]
[1.90936249e+108]
[9.22126001e+107]
[4.44862622e+107]
[2.14385709e+107]
[1.03204947e+107]
[4.96295639e+106]
[2.38405441e+106]
[1.14400553e+106]
[5.48373975e+105]
[2.62580694e+105]
[1.25599088e+105]
[6.00134239e+104]
[2.86450144e+104]
[1.36580561e+104]
[6.50531658e+103]
[3.09519607e+103]
[1.47112157e+103]
[6.98473795e+102]
[3.31278546e+102]
[1.56956255e+102]
[7.42859275e+101]
[3.51218642e+101]
[1.65879277e+101]
[7.82619664e+100]
[3.68853494e+100]
[1.736609e+100]
[8.1676228e+99]
[3.83738391e+99]
[1.80103127e+99]
[8.44411536e+98]
[3.95489023e+98]
[1.85038694e+98]
[8.64846412e+97]
[4.03798039e+97]
[1.88338312e+97]
[8.77531796e+96]
[4.08448433e+96]
[1.89916307e+96]
[8.82141787e+95]
[4.09322965e+95]
[1.89734309e+95]
[8.78573561e+94]
[4.06409034e+94]
[1.87802773e+94]
[8.66950954e+93]
[3.99798717e+93]
[1.84180251e+93]
[8.47617622e+92]
[3.8

[6.29304061e-89]
[2.00959681e-89]
[6.41281916e-90]
[2.04494171e-90]
[6.51635852e-91]
[2.0750154e-91]
[6.60283148e-92]
[2.09957723e-92]
[6.67154567e-93]
[2.11843069e-93]
[6.72195278e-94]
[2.13142607e-94]
[6.75365584e-95]
[2.13846245e-95]
[6.76641447e-96]
[2.13948894e-96]
[6.76014779e-97]
[2.13450534e-97]
[6.73493526e-98]
[2.12356199e-98]
[6.69101514e-99]
[2.10675895e-99]
[6.62878084e-100]
[2.0842445e-100]
